In [26]:
# -*- coding: utf-8 -*-
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np # 数据处理的库 numpy
import argparse
import imutils
import time
import dlib
import cv2
 
 
def eye_aspect_ratio(eye):
    # 垂直眼标志（X，Y）坐标
    A = dist.euclidean(eye[1], eye[5])# 计算两个集合之间的欧式距离
    B = dist.euclidean(eye[2], eye[4])
    # 计算水平之间的欧几里得距离
    # 水平眼标志（X，Y）坐标
    C = dist.euclidean(eye[0], eye[3])
    # 眼睛长宽比的计算
    ear = (A + B) / (2.0 * C)
    # 返回眼睛的长宽比
    return ear
 
 
# 定义两个常数
# 眼睛长宽比
# 闪烁阈值
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3
# 初始化帧计数器和眨眼总数
COUNTER = 0
TOTAL = 0
 
# 初始化DLIB的人脸检测器（HOG），然后创建面部标志物预测
print("[INFO] loading facial landmark predictor...")
# 第一步：使用dlib.get_frontal_face_detector() 获得脸部位置检测器
detector = dlib.get_frontal_face_detector()
# 第二步：使用dlib.shape_predictor获得脸部特征位置检测器
predictor = dlib.shape_predictor('D:/myworkspace/JupyterNotebook/fatigue_detecting/model/shape_predictor_68_face_landmarks.dat')
 
# 第三步：分别获取左右眼面部标志的索引
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# 第四步：打开cv2 本地摄像头
cap = cv2.VideoCapture(0)
 
# 从视频流循环帧
while True:
    # 第五步：进行循环，读取图片，并对图片做维度扩大，并进灰度化
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=720)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 第六步：使用detector(gray, 0) 进行脸部位置检测
    rects = detector(gray, 0)
    
    # 第七步：循环脸部位置信息，使用predictor(gray, rect)获得脸部特征位置的信息
    for rect in rects:
        shape = predictor(gray, rect)
        
        # 第八步：将脸部特征信息转换为数组array的格式
        shape = face_utils.shape_to_np(shape)
        
        # 第九步：提取左眼和右眼坐标
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        
        # 第十步：构造函数计算左右眼的EAR值，使用平均值作为最终的EAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
 
        # 第十一步：使用cv2.convexHull获得凸包位置，使用drawContours画出轮廓位置进行画图操作
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
 
        # 第十二步：进行画图操作，用矩形框标注人脸
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)    
 
        '''
            分别计算左眼和右眼的评分求平均作为最终的评分，如果小于阈值，则加1，如果连续3次都小于阈值，则表示进行了一次眨眼活动
        '''
        # 第十三步：循环，满足条件的，眨眼次数+1
        if ear < EYE_AR_THRESH:# 眼睛长宽比：0.2
            COUNTER += 1
           
        else:
            # 如果连续3次都小于阈值，则表示进行了一次眨眼活动
            if COUNTER >= EYE_AR_CONSEC_FRAMES:# 阈值：3
                TOTAL += 1
            # 重置眼帧计数器
            COUNTER = 0
            
        # 第十四步：进行画图操作，68个特征点标识
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
            
        # 第十五步：进行画图操作，同时使用cv2.putText将眨眼次数进行显示
        cv2.putText(frame, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "COUNTER: {}".format(COUNTER), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (450, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

 
    print('眼睛实时长宽比:{:.2f} '.format(ear))
    if TOTAL >= 50:
        cv2.putText(frame, "SLEEP!!!", (200, 200),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
    # 窗口显示 show with opencv
    cv2.imshow("Frame", frame)
    
    # if the `q` key was pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# 释放摄像头 release camera
cap.release()
# do a bit of cleanup
cv2.destroyAllWindows()


[INFO] loading facial landmark predictor...
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.22 
眼睛实时长宽比:0.42 
眼睛实时长宽比:0.35 
眼睛实时长宽比:0.32 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.32 
眼睛实时长宽比:0.34 
眼睛实时长宽比:0.35 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.32 
眼睛实时长宽比:0.37 
眼睛实时长宽比:0.35 
眼睛实时长宽比:0.34 
眼睛实时长宽比:0.33 
眼睛实时长宽比:0.35 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.29 
眼睛实时长宽比:0.29 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.35 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.27 
眼睛实时长宽比:0.29 
眼睛实时长宽比:0.21 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.19 
眼睛实时长宽比:0.28 
眼睛实时

眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.25 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.35 
眼睛实时长宽比:0.38 
眼睛实时长宽比:0.35 
眼睛实时长宽比:0.41 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.36 
眼睛实时长宽比:0.30 
眼睛实时长宽比:0.29 
眼睛实时长宽比:0.27 
眼睛实时长宽比:0.29 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.25 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.27 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽比:0.24 
眼睛实时长宽

眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.26 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.31 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.28 
眼睛实时长宽比:0.28 
眼睛实时长宽

眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽

眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽

眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽

眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽比:0.46 
眼睛实时长宽

In [17]:
from scipy.spatial import distance as dis
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import pyglet
import argparse
import imutils
import time
import dlib
import cv2

#计算嘴的长宽比,euclidean(u, v, w=None)用于计算两点的欧几里得距离
def mouthRatio(mouth):
    left=dis.euclidean(mouth[2],mouth[10])
    mid=dis.euclidean(mouth[3],mouth[9])
    right=dis.euclidean(mouth[4],mouth[8])
    horizontal=dis.euclidean(mouth[0],mouth[6])
    return 10.0*horizontal/(3.0*left+4.0*mid+3.0*right)

#计算眼睛的长宽比
def eyesRatio(eye):
    left = dis.euclidean(eye[1], eye[5])
    right = dis.euclidean(eye[2], eye[4])
    horizontal = dis.euclidean(eye[0], eye[3])
    return 2.0*horizontal/(left+right)

#创建一个解析对象，向该对象中添加关注的命令行参数和选项，然后解析
ap = argparse.ArgumentParser()
ap.add_argument("-w", "--webcam", type=int, default=0)
args = vars(ap.parse_args())

#眼睛长宽比的阈值，如果超过这个值就代表眼睛长/宽大于采集到的平均值，默认已经"闭眼"
eyesRatioLimit=0
#数据采集的计数，采集30次然后取平均值
collectCount=0
#用于数据采集的求和
collectSum=0
#是否开始检测
startCheck=False

#统计"闭眼"的次数
eyesCloseCount=0

#初始化dlib
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("D:/myworkspace/JupyterNotebook/fatigue_detecting/model/shape_predictor_68_face_landmarks.dat")

#获取面部各器官的索引
#左右眼
(left_Start,left_End)=face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_Start,right_End)=face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
#嘴
(leftMouth,rightMouth)=face_utils.FACIAL_LANDMARKS_IDXS['mouth']
#下巴
(leftJaw,rightJaw)=face_utils.FACIAL_LANDMARKS_IDXS['jaw']
#鼻子
(leftNose,rightNose)=face_utils.FACIAL_LANDMARKS_IDXS['nose']
#左右眉毛
(left_leftEyebrow,left_rightEyebrow)=face_utils.FACIAL_LANDMARKS_IDXS['left_eyebrow']
(right_leftEyebrow,right_rightEyebrow)=face_utils.FACIAL_LANDMARKS_IDXS['right_eyebrow']

#开启视频线程，延迟2秒钟
vsThread=VideoStream(src=args["webcam"]).start()
time.sleep(2.0)

#循环检测
while True:
    #对每一帧进行处理，设置宽度并转化为灰度图
    frame = vsThread.read()
    frame = imutils.resize(frame, width=720)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #检测灰度图中的脸
    faces = detector(img, 0)
    for k in faces:
        #确定面部区域的面部特征点，将特征点坐标转换为numpy数组
        shape = predictor(img, k)
        shape = face_utils.shape_to_np(shape)

        #左右眼
        leftEye = shape[left_Start:left_End]
        rightEye = shape[right_Start:right_End]
        leftEyesVal = eyesRatio(leftEye)
        rightEyesVal = eyesRatio(rightEye)
        #凸壳
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        #绘制轮廓
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        #取两只眼长宽比的的平均值作为每一帧的计算结果
        eyeRatioVal = (leftEyesVal + rightEyesVal) / 2.0

        #嘴
        mouth=shape[leftMouth:rightMouth]
        mouthHull=cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

        #鼻子
        nose=shape[leftNose:rightNose]
        noseHull=cv2.convexHull(nose)
        cv2.drawContours(frame, [noseHull], -1, (0, 255, 0), 1)

        #下巴
        jaw=shape[leftJaw:rightJaw]
        jawHull=cv2.convexHull(jaw)
        cv2.drawContours(frame, [jawHull], -1, (0, 255, 0), 1)

        #左眉毛
        leftEyebrow=shape[left_leftEyebrow:left_rightEyebrow]
        leftEyebrowHull=cv2.convexHull(leftEyebrow)
        cv2.drawContours(frame, [leftEyebrowHull], -1, (0, 255, 0), 1)

        #右眉毛
        rightEyebrow=shape[right_leftEyebrow:right_rightEyebrow]
        rightEyebrowHull=cv2.convexHull(rightEyebrow)
        cv2.drawContours(frame, [rightEyebrowHull], -1, (0, 255, 0), 1)

        if collectCount<30:
            collectCount+=1
            collectSum+=eyeRatioVal
            cv2.putText(frame, "DATA COLLECTING", (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            startCheck=False
        else:
            if not startCheck:
                eyesRatioLimit=collectSum/(1.0*30)
                print('眼睛长宽比均值',eyesRatioLimit)
            startCheck=True

        if startCheck:
            #如果眼睛长宽比大于之前检测到的阈值，则计数，闭眼次数超过50次则认为已经"睡着"
            if eyeRatioVal > eyesRatioLimit:
                eyesCloseCount += 1
                if eyesCloseCount >= 50:
                    cv2.putText(frame, "SLEEP!!!", (580, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            else:  
                eyesCloseCount = 0
            print('眼睛实时长宽比:{:.2f} '.format(eyeRatioVal))
            #眼睛长宽比
            cv2.putText(frame, "EYES_RATIO: {:.2f}".format(eyeRatioVal), (20, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 160, 0), 2)
            #闭眼次数
            cv2.putText(frame,"EYES_COLSE: {}".format(eyesCloseCount),(320,30),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,160,0),2)

            #通过检测嘴的长宽比检测有没有打哈欠，后来觉得没什么卵用
            #cv2.putText(frame,"MOUTH_RATIO: {:.2f}".format(mouthRatio(mouth)),(30, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    #停止
    if key == ord("S"):  break

cv2.destroyAllWindows()
vsThread.stop()


usage: ipykernel_launcher.py [-h] [-w WEBCAM]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\CUNGU\AppData\Roaming\jupyter\runtime\kernel-303ab4b2-729c-46f6-a425-07bccbcfdf31.json


SystemExit: 2

D:\mydownload\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
